<a href="https://colab.research.google.com/github/nashrah-18/YourIndianTranslatorv1/blob/main/FinalEngToHin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Checking if GPU is running or not
!nvidia-smi

Mon Feb 17 09:59:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#LETS FIRST PUSH MY CREATED DATASET INTO MY HUGGINGFACE ACCOUNT
!pip install datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00


In [3]:
#Load dataset locally
from datasets import load_dataset

dataset = load_dataset('json', data_files="/content/eng-hindi.json")

print(dataset['train'][4])

Generating train split: 0 examples [00:00, ? examples/s]

{'english_text': 'How are you?', 'hindi_translation': 'क्या हाल चाल है?', 'topic': 'Language Basics and Phrases'}


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
dataset.push_to_hub('nashrah18/indiantranslator')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/390 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/nashrah18/indiantranslator/commit/cbce10c43b6b97628f391349a225170becf6d3a7', commit_message='Upload dataset', commit_description='', oid='cbce10c43b6b97628f391349a225170becf6d3a7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/nashrah18/indiantranslator', endpoint='https://huggingface.co', repo_type='dataset', repo_id='nashrah18/indiantranslator'), pr_revision=None, pr_num=None)

In [6]:
#DATA TOKENIZING AND MODEL TRAINING
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.8 MB/s eta 0:00:00


In [7]:
#Imports
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
#Choose the model
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [9]:
#Load our dataset that we just pushed so we can train it
raw_datasets = load_dataset("nashrah18/indiantranslator")

train-00000-of-00001.parquet:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/145 [00:00<?, ? examples/s]

In [10]:
#Check the dataset
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['english_text', 'hindi_translation', 'topic'],
        num_rows: 145
    })
})

In [11]:
#Check specific column
raw_datasets['train'][100]

{'english_text': 'What are the common online scams targeting tourists in India?',
 'hindi_translation': 'यार, इंडिया में टूरिस्ट्स पर आम ऑनलाइन स्कैम्स कौन से हैं?',
 'topic': 'Avoiding Scams and Petty Crimes'}

In [12]:
#PREPROCESSING OUR DATA
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [13]:
#Check tokenizer
tokenizer("Hello, this is a sentence!")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
#Checking two sentences at the same time
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[12110, 2, 90, 23, 19, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [15]:
#Check whether it does tokenizing to hindi sentences
with tokenizer.as_target_tokenizer():
    print(tokenizer(["यार, इंडिया में टूरिस्ट्स पर आम ऑनलाइन स्कैम्स कौन से हैं?"]))

{'input_ids': [[13005, 2, 28144, 11, 44122, 10972, 1307, 33, 2152, 4871, 18784, 16284, 1307, 391, 12, 28, 22, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
#Convert it into numbers
max_input_length=128
max_target_length=128

source_lang="english_text" #your dataset column name
target_lang="hindi_translation" #your dataset column name

def preprocess_function(examples):
    # Access the correct columns using column names
    inputs = examples[source_lang]
    targets = examples[target_lang]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[12110, 0], [172, 16, 213, 22, 0]], 'attention_mask': [[1, 1], [1, 1, 1, 1, 1]], 'labels': [[4788, 351, 26090, 0], [65, 1218, 1110, 22, 0]]}

In [18]:
#Map tokens
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/145 [00:00<?, ? examples/s]

In [19]:
#Load model
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [20]:
batch_size = 16
learning_rate = 5e-5
weight_decay = 0.01
num_train_epochs = 10 #keep epochs high

In [21]:
#Passes data to your model batch wise
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [22]:
#Passes data with parameters
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [23]:
#Load Training data
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [24]:
#Compile your model
#Deploy your optimizer
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [25]:
#Feed my dataset to my model
model.fit(train_dataset, epochs=10)

Epoch 1/10
9/9 [==============================] - 34s 160ms/step - loss: 3.1178
Epoch 2/10
9/9 [==============================] - 1s 151ms/step - loss: 2.0750
Epoch 3/10
9/9 [==============================] - 2s 191ms/step - loss: 1.5591
Epoch 4/10
9/9 [==============================] - 2s 165ms/step - loss: 1.1989
Epoch 5/10
9/9 [==============================] - 1s 152ms/step - loss: 0.9421
Epoch 6/10
9/9 [==============================] - 1s 152ms/step - loss: 0.7435
Epoch 7/10
9/9 [==============================] - 1s 151ms/step - loss: 0.5912
Epoch 8/10
9/9 [==============================] - 1s 152ms/step - loss: 0.4796
Epoch 9/10
9/9 [==============================] - 1s 154ms/step - loss: 0.4030
Epoch 10/10
9/9 [==============================] - 1s 148ms/step - loss: 0.3448


In [26]:
#Save the model
model.save_pretrained("indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


In [27]:
#Tokenize model
tokenizer.save_pretrained("indian_translatorv1")

('indian_translatorv1/tokenizer_config.json',
 'indian_translatorv1/special_tokens_map.json',
 'indian_translatorv1/vocab.json',
 'indian_translatorv1/source.spm',
 'indian_translatorv1/target.spm',
 'indian_translatorv1/added_tokens.json')

In [28]:
#Push our model indian_translatorv1
model.push_to_hub("nashrah18/indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [29]:
#Tokenize our model
tokenizer.push_to_hub("nashrah18/indian_translatorv1")

README.md:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/nashrah18/indian_translatorv1/commit/de71625178afd7575fbf7a18173d619da0fd8641', commit_message='Upload tokenizer', commit_description='', oid='de71625178afd7575fbf7a18173d619da0fd8641', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nashrah18/indian_translatorv1', endpoint='https://huggingface.co', repo_type='model', repo_id='nashrah18/indian_translatorv1'), pr_revision=None, pr_num=None)

#MODEL TESTING

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at indian_translatorv1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [32]:
#Tokenize output
input_text  = "Which are the safest areas here where women can stay?"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
#Change numbers into text
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

यहाँ पे महिलाओं के लिए सबसे सुरक्षित जगहें कौन सी हैं भाई??
